In [10]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="mistral",
    temperature=0,
)


import duckdb

conn = duckdb.connect("data/magpie.db")

cursor = conn.cursor()
#conn.close() 


In [16]:
 
conn.close() 

NameError: name 'conn' is not defined

In [5]:
conn.close() 

In [11]:
# Erstelle neue Tabelle mit geändertem Datentyp
cursor.execute("""
    CREATE TABLE datensatz_drittmittel_hochschule_new AS
    SELECT 
        CAST(jahr AS INTEGER) AS jahr,
        * EXCLUDE jahr
    FROM datensatz_drittmittel_hochschule;
""")

# Lösche die alte Tabelle
cursor.execute("DROP TABLE datensatz_drittmittel_hochschule;")

# Benenne die neue Tabelle um
cursor.execute("""
    ALTER TABLE datensatz_drittmittel_hochschule_new
    RENAME TO datensatz_drittmittel_hochschule;
""")


In [12]:
schema_dict = {}

# Tabelle 'datensatz_fue_erhebung' analysieren
table_name = "datensatz_drittmittel_hochschule"
schema_dict[table_name] = cursor.execute(f"DESCRIBE {table_name};").fetchall()

schema_dict

{'datensatz_drittmittel_hochschule': [('jahr',
   'INTEGER',
   'YES',
   None,
   None,
   None),
  ('id', 'INTEGER', 'YES', None, None, None),
  ('Variable', 'VARCHAR', 'YES', None, None, None),
  ('Zeit', 'TIMESTAMP', 'YES', None, None, None),
  ('Hochschule', 'VARCHAR', 'YES', None, None, None),
  ('Wert', 'VARCHAR', 'YES', None, None, None),
  ('Einheit', 'VARCHAR', 'YES', None, None, None),
  ('Quelle', 'VARCHAR', 'YES', None, None, None)]}

In [13]:
import pandas as pd

# Tabelle 'datensatz_fue_erhebung' in ein Pandas-DataFrame laden
query = "SELECT * FROM datensatz_drittmittel_hochschule;"
df = pd.read_sql(query, conn)  # conn ist die Verbindung zu deiner DuckDB
 
# DataFrame anzeigen
print(df)

C:\Users\Hueck\AppData\Local\Temp\ipykernel_42596\1778066478.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)  # conn ist die Verbindung zu deiner DuckDB


       jahr      id                 Variable       Zeit  \
0      2006   30746     Drittmittel vom Bund 2006-01-01   
1      2007   30747     Drittmittel vom Bund 2007-01-01   
2      2008   30748     Drittmittel vom Bund 2008-01-01   
3      2010   30750     Drittmittel vom Bund 2010-01-01   
4      2011   30751     Drittmittel vom Bund 2011-01-01   
...     ...     ...                      ...        ...   
87855  2021  118392  Drittmittel von der DFG 2021-01-01   
87856  2021  118406  Drittmittel von der DFG 2021-01-01   
87857  2021  118418  Drittmittel von der DFG 2021-01-01   
87858  2021  118475  Drittmittel von der DFG 2021-01-01   
87859  2021  118595  Drittmittel von der DFG 2021-01-01   

                                              Hochschule  \
0                                     Universität Kassel   
1                                     Universität Kassel   
2                                     Universität Kassel   
3                                     Universität K

In [19]:
import re

def simple_prompt_engineering(question, schema, llm, conn):

    direct_prompt = f"""
    Given an input question, create a syntactically correct SQL query to
    run in DuckDB. The database is defined by the following schema:

    {schema}

    Only use the following tables:
    {conn.execute("SHOW TABLES").fetchall()}

    Don't use more columns than strictly necessary. Be careful to not
    query for columns that do not exist. Also, pay attention to which
    column is in which table. Please think carefully before you answer.

    Return only a SQL query and nothing else.

    Question: """

    response = llm.invoke(direct_prompt + question)

    print(response.content)
    result = str(conn.execute(response.content).fetchall()[0][0])
    print(f"Answer: {result}")

simple_prompt_engineering(
    "Wie hoch sind die Drittmittel der Universität Kassel im Jahr 2011", schema_dict, llm, conn
)

 SELECT Wert FROM datensatz_drittmittel_hochschule WHERE Hochschule = 'Universität Kassel' AND Jahr = 2011;
Answer: 12199.8060000000004947651


# Langchain Resolution

In [1]:
import duckdb
import pandas as pd

# Verbindung zur Datenbank herstellen
conn = duckdb.connect("data/drittmittel_hs.db")
cursor = conn.cursor()

# SQL-Abfrage ausführen
query = "SELECT * FROM datensatz_drittmittel_hochschule"
df = cursor.execute(query).fetchdf()

# Ergebnisse anzeigen
print(df)

# Verbindung schließen
conn.close()


       jahr      id                 Variable       Zeit  \
0      2006   30746     Drittmittel vom Bund 2006-01-01   
1      2007   30747     Drittmittel vom Bund 2007-01-01   
2      2008   30748     Drittmittel vom Bund 2008-01-01   
3      2010   30750     Drittmittel vom Bund 2010-01-01   
4      2011   30751     Drittmittel vom Bund 2011-01-01   
...     ...     ...                      ...        ...   
87855  2021  118392  Drittmittel von der DFG 2021-01-01   
87856  2021  118406  Drittmittel von der DFG 2021-01-01   
87857  2021  118418  Drittmittel von der DFG 2021-01-01   
87858  2021  118475  Drittmittel von der DFG 2021-01-01   
87859  2021  118595  Drittmittel von der DFG 2021-01-01   

                                              Hochschule  \
0                                     Universität Kassel   
1                                     Universität Kassel   
2                                     Universität Kassel   
3                                     Universität K

In [9]:
# Verbindung mit langchain-Tool
from langchain_ollama import ChatOllama
import re

llm = ChatOllama(
    model="mistral",
    temperature=0,
)


import duckdb
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("duckdb:///data/drittmittel_hs.db")

c:\Users\Hueck\miniconda3\envs\RAG_Pipeline\lib\site-packages\duckdb_engine\__init__.py:174: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


In [10]:
 
print(db.get_usable_table_names())

['datensatz_drittmittel_hochschule']


In [12]:
def with_a_little_langchain(question, verbose=False):
    direct_prompt = f"""
    Given an input question, create a syntactically correct SQL query to
    run in DuckDB. The database is defined by the following schema:

    {db.get_table_info()}

    Only use the following tables:
    {db.get_usable_table_names()}

    Pay attention to use only the column names you can see in the tables
    below. Be careful to not query for columns that do not exist. Also, pay
    attention to which column is in which table. Please carefully think
    before you answer.

    Question: 
    """

    response = llm.invoke(direct_prompt + question)

    if verbose:
        print(response)

    print(re.search("(SELECT.*);", response.content.replace("\n", " ")).group(1))
    print(
        db.run(re.search("(SELECT.*);", response.content.replace("\n", " ")).group(1))
    )

with_a_little_langchain("Wie hoch waren die Drittmittel von Stiftungen der Universität Kassel im Jahr 2008?", verbose=True)

content=' To find the third-party funds from foundations of the University of Kassel in the year 2008, you can use the following SQL query:\n\n```sql\nSELECT Wert AS "Third-Party Funds"\nFROM datensatz_drittmittel_hochschule\nWHERE Hochschule = \'Universität Kassel\' AND Jahr = 2008 AND Variable = \'Drittmittel von Stiftungen\';\n```\n\nThis query selects the "Third-Party Funds" (Wert) column from the `datensatz_drittmittel_hochschule` table where the university is \'Universität Kassel\', the year is 2008, and the variable is \'Drittmittel von Stiftungen\'.' additional_kwargs={} response_metadata={'model': 'mistral', 'created_at': '2024-12-09T10:03:22.0735483Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3695776100, 'load_duration': 6034900, 'prompt_eval_count': 568, 'prompt_eval_duration': 347853000, 'eval_count': 181, 'eval_duration': 3339410000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-32c18b00-f7cd-4dda-9b58-d8db57e0fd5d-0'